In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
from signal_preprocessing import MedianFilter, SGFilter, MedianBaselineCorrector
import pandas as pd

In [2]:
def load_data(filename):
    with open(os.path.join('data', filename)) as f:
        content = json.load(f)
        data = content['data']
        timestamps = content['timestamps']
        df = pd.DataFrame(data, index=timestamps['ear'])
        df = -df
        df.index -= df.index[0]
        fps = len(df)/(df.index[-1]-df.index[0])
        print('fps:', fps)
        df['ear+adr'] = df['ear'] + df['adr']
        df['ear+adr+vd'] = df['ear'] / df['ear']/np.linalg.norm(df['ear']) + \
            df['adr']/np.linalg.norm(df['adr']) +\
            df['vd']/np.linalg.norm(df['vd'])
    return df

In [3]:
from scipy import signal


class BasicPeakFinder():
    def __init__(self, time_window_length):
        self.time_window_length = time_window_length

    @staticmethod
    def slide_time_windows(timestamps, window_length):
        '''Slide time windows and return tuples (start_index, end_index) 
        for which timestamps[start_index:end_index] is in the window_length'''

        if len(timestamps) == 0:
            return 

        start_index = 0
        end_index = 1
        prev_end_index = 0

        while end_index < len(timestamps):
            # move end_index to the end
            while end_index < len(timestamps) and timestamps[end_index] - timestamps[start_index] <= window_length:
                end_index += 1

            if end_index > prev_end_index:
                yield (start_index, end_index)
                prev_end_index = end_index

            # step start_index
            start_index += 1
    
    def find(self, x, y):
        peaks = []
        props = {}
        last_peak_index = -1
        for start_index, end_index in BasicPeakFinder.slide_time_windows(x, self.time_window_length):
            y_window = y[start_index:end_index]
            peak_index = start_index + np.argmax(y_window)
            if peak_index > last_peak_index:
                peaks.append(peak_index)
                last_peak_index = peak_index
        return (peaks, props)


class ScipyPeakFinder():
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def find(self, x, y):
        return signal.find_peaks(y, **self.kwargs)


In [11]:
from blink_detector import PeakFinder, plot_peaks
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
import os

@interact(medfiltsize=widgets.IntSlider(min=1, max=15, step=2, value=5), 
          sg_window_length=widgets.IntSlider(min=1, max=15, step=2, value=7), 
          sg_polyorder=widgets.IntSlider(min=0, max=15, step=1, value=2),
          med_bc_window_length=widgets.IntSlider(min=1, max=33, step=2, value=31),
          pf_prominence=widgets.FloatSlider(min=-0.1, max=1, step=0.01, value=0.03),
          pf_min_width=widgets.IntSlider(min=1, max=30, value=3),
          pf_max_width=widgets.IntSlider(min=1, max=30, value=16),
          pf_wlen=widgets.IntSlider(min=3, max=66, value=29),
        )
def plot(medfiltsize, sg_window_length, sg_polyorder, med_bc_window_length, 
         pf_prominence, pf_wlen, pf_min_width, pf_max_width, column=['adr', 'ear', 'vd'], 
         sg_filter=True, med_filter=False, baseline_correction=True,
         filename=os.listdir('data') ):
    # create filters
    preps = []
    if med_filter:
        preps.append(MedianFilter(filtersize=medfiltsize))
    if sg_filter:
        preps.append(SGFilter(sg_window_length, sg_polyorder))
    if baseline_correction:
        preps.append(MedianBaselineCorrector(med_bc_window_length))

    # load data
    df = load_data(filename)
    df = df.iloc[:60]

    # get the data
    y = df[column].values[:]
    x = df.index[:]
    
    fig = plt.figure(0, figsize=(len(df)/10, 6))
        
    # preprocess data
    for p in preps:
        y = p.apply(y)
    
    
    # plot data
    plt.plot(x, y, label=column)

    # plot peaks
    if pf_prominence < 0:
        pf_prominence = np.std(y) * 3
        print('pf_prominance:', pf_prominence)

    peak_finder = PeakFinder(prominence=pf_prominence, wlen=pf_wlen, rel_height=0.5, 
                             width=(pf_min_width, pf_max_width))
    
    peaks, props = peak_finder.find(x, y)
    print(peaks, props)
    plot_peaks(peaks, props, x, y)
    print("Number of peaks:", len(peaks))

    plt.legend()
    plt.xticks(range(int(x[-1] + 1)))
    


interactive(children=(IntSlider(value=5, description='medfiltsize', max=15, min=1, step=2), IntSlider(value=7,…